In [1]:
# Importing necessary libraries
import torch
from transformers import (
    BertForQuestionAnswering,
    BertTokenizerFast,
)
from scipy.special import softmax
import plotly.express as px
import pandas as pd
import numpy as np


In [2]:
# Defining the context and the question
context = "The giraffe is a large African hoofed mammal belonging to the genus Giraffa. It is the tallest living terrestrial animal and the largest ruminant on Earth. Traditionally, giraffes were thought to be one species, Giraffa camelopardalis, with nine subspecies. Most recently, researchers proposed dividing them into up to eight extant species due to new research into their mitochondrial and nuclear DNA, as well as morphological measurements. Seven other extinct species of Giraffa are known from the fossil record."
question = "How many giraffe species are there?"


In [3]:
# Defining the model name and loading the tokenizer and the model
model_name = "deepset/bert-base-cased-squad2"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Tokenizing the context and the question
inputs = tokenizer(question, context, return_tensors="pt")
tokenizer.tokenize(context)


['The',
 'g',
 '##ira',
 '##ffe',
 'is',
 'a',
 'large',
 'African',
 'ho',
 '##of',
 '##ed',
 'ma',
 '##mmal',
 'belonging',
 'to',
 'the',
 'genus',
 'G',
 '##ira',
 '##ff',
 '##a',
 '.',
 'It',
 'is',
 'the',
 'tallest',
 'living',
 'terrestrial',
 'animal',
 'and',
 'the',
 'largest',
 'r',
 '##umi',
 '##nant',
 'on',
 'Earth',
 '.',
 'Traditionally',
 ',',
 'g',
 '##ira',
 '##ffe',
 '##s',
 'were',
 'thought',
 'to',
 'be',
 'one',
 'species',
 ',',
 'G',
 '##ira',
 '##ff',
 '##a',
 'came',
 '##lop',
 '##ard',
 '##alis',
 ',',
 'with',
 'nine',
 'subspecies',
 '.',
 'Most',
 'recently',
 ',',
 'researchers',
 'proposed',
 'dividing',
 'them',
 'into',
 'up',
 'to',
 'eight',
 'extant',
 'species',
 'due',
 'to',
 'new',
 'research',
 'into',
 'their',
 'mitochondrial',
 'and',
 'nuclear',
 'DNA',
 ',',
 'as',
 'well',
 'as',
 'm',
 '##or',
 '##phological',
 'measurements',
 '.',
 'Seven',
 'other',
 'extinct',
 'species',
 'of',
 'G',
 '##ira',
 '##ff',
 '##a',
 'are',
 'known',
 

In [5]:
# Running the model and getting the start and end scores
with torch.no_grad():
    outputs = model(**inputs)
start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]


In [6]:
# Creating a dataframe with the scores and plotting them
scores_df = pd.DataFrame({
    "Token Position": list(range(len(start_scores))) * 2,
    "Score": list(start_scores) + list(end_scores),
    "Score Type": ["Start"] * len(start_scores) + ["End"] * len(end_scores),
})
px.bar(scores_df, x="Token Position", y="Score", color="Score Type", barmode="group", title="Start and End Scores for Tokens")


In [9]:
# Getting the answer from the model
start_idx = np.argmax(start_scores)
end_idx = np.argmax(end_scores)
answer_ids = inputs.input_ids[0][start_idx: end_idx + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
answer = tokenizer.convert_tokens_to_string(answer_tokens)
answer

'eight'

In [10]:
# Part 2
# Defining a function to predict the answer to a question given a context
def predict_answer(context, question):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]
    start_idx = np.argmax(start_scores)
    end_idx = np.argmax(end_scores)
    confidence_score = (start_scores[start_idx] + end_scores[end_idx]) /2
    answer_ids = inputs.input_ids[0][start_idx: end_idx + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    if answer != tokenizer.cls_token:
        return answer, confidence_score
    return None, confidence_score


In [11]:
# Defining a new context and predicting answers to some questions
context = """Coffee is a beverage prepared from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due to its caffeine content. It has the highest sales in the world market for hot drinks.[2][unreliable source?]
...
"""
len(tokenizer.tokenize(context))
predict_answer(context, "What is coffee?")
predict_answer(context, "What are the most common coffee beans?")
predict_answer(context, "How can I make ice coffee?")
predict_answer(context[4000:], "How many people are dependent on coffee for their income?")


(None, 0.9988429546356201)

In [12]:
# Defining a function to chunk sentences
def chunk_sentences(sentences, chunk_size, stride):
    chunks = []
    num_sentences = len(sentences)
    for i in range(0, num_sentences, chunk_size - stride):
        chunk = sentences[i: i + chunk_size]
        chunks.append(chunk)
    return chunks




In [14]:
sentences = [
    "Sentence 1.",
    "Sentence 2.",
    "Sentence 3.",
    "Sentence 4.",
    "Sentence 5.",
    "Sentence 6.",
    "Sentence 7.",
    "Sentence 8.",
    "Sentence 9.",
    "Sentence 10."
]

chunked_sentences = chunk_sentences(sentences, chunk_size=3, stride=1)


questions = ["What is coffee?", "What are the most common coffee beans?", "How can I make ice coffee?", "How many people are dependent on coffee for their income?"]

answers = {}

for chunk in chunked_sentences:
    context = "\n".join(chunk)
    for question in questions:
        answer, score = predict_answer(context, question)

        if answer:
            if question not in answers:
                answers[question] = (answer, score)
            else:
                if score > answers[question][1]:
                    answers[question] = (answer, score)

answers

{}